# INMARSAT May Bill 2021

## Import Libraries

In [1]:
import pandas as pd

import numpy as np

import re

import os

## Cleaning Data: 
1. Uploading Invoice, 2. Removing NaNs, 3. Renaming Columns Needed, 4. Filtering Totals, 5. Resetting Index To Current Filtered Rows, 6. Removing 'Unamed' Columns, 7. Getting ID Numbers For New 'IDs' Column, 8. Getting Amounts Creating New 'Amounts Due' Column, 9. Turning Amounts Due from Strings of Text into Numbers, 10. Removing 'ID' and 'Totals' Column, 11. Saving Version to .csv, 12. Checking My Work, 13. Checking Amounts Due Totals

In [2]:
df = pd.read_csv('May_2021_Invoice.csv') #1. Uploading Invoice

df = df.fillna("") #2. Removing NaNs

df = df.rename(columns={"Unnamed: 8": "ID", "Unnamed: 20": "Totals"}) #3. Renaming Columns Needed
df = df.loc[df['ID'].str.contains('^to[a-z]*', flags=re.I, regex=True)] #4. Filtering Totals
df = df.reset_index(drop=True) #5. Resetting Index To Current Filtered Rows

df = df.loc[:, ~df.columns.str.contains('^Unnamed')] #6. Removing 'Unamed' Columns
def get_ID(ID):
    return ID.split(' ')[2] #7. Getting ID Numbers For New 'IDs' Column

df['IDs'] = df['ID'].apply(lambda x: get_ID(x)) #7. Getting ID Numbers For New 'IDs' Column

df['Amounts Due'] = df['Totals'].apply(lambda x: x.split('$')[1]) #8. Getting Amounts Creating New 'Amounts Due' Column

df['Amounts Due'] = pd.to_numeric(df['Amounts Due']) #9. Turning Amounts Due from Strings of Text into Numbers

df = df.drop(columns=['ID', 'Totals']) #10. Removing 'ID' and 'Totals' Column

df.to_excel('INMARSAT May 2021.xlsx', index=False) #11. Saving Version to CSV

df

,Bill To:,IDs,Amounts Due
0,10/11/2019 -,000000000100,13.79
1,,000000000101,13.79
2,,000000000102,13.79
3,,000000000103,13.79
4,,000000000104,13.79
...,...,...,...
522,,881651461225,15.44
523,,881651477444,25.19
524,,881651477445,44.94
525,,881651477579,15.44


In [3]:
df.sum(axis = 0, skipna = True) #13. Checking Amounts Due Totals

Bill To:       10/11/2019 -10/11/2019 -10/11/2019 -10/11/2019...
IDs            0000000001000000000001010000000001020000000001...
Amounts Due                                              14658.2
dtype: object

## Merging .csv Files Based on ID Numbers
1. Defining Our Previous Notebook Saved, 2. PI and ID Number Data, 3. Uploading Data, 4. Printing to see our columns, 5. Merging Our Files Based on ID Number, 6. Saving Version to .csv, 7. Checking My Work, 8. Checking Amounts Due Totals

In [5]:
initial_workbook = 'INMARSAT May 2021.xlsx' #1. Defining Our Previous Notebook Saved
info_workbook = 'Active DF COMPLETE 26 May.xlsx' #2. Defining PI and ID Number Dataframe

df_initial = pd.read_excel(initial_workbook) #3. Uploading Data
df_info = pd.read_excel(info_workbook) #3. Uploading Data

print(df_initial.columns) #4. Printing to see our columns
print(df_info.columns) #4. Printing to see our columns


Index(['Bill To:', 'IDs', 'Amounts Due'], dtype='object')
Index(['IDs', 'ICCID', 'MSISDN-C', 'Type', 'PI'], dtype='object')


In [6]:
df_3 = pd.merge(df_initial, df_info[['IDs','ICCID','MSISDN-C','PI','Type']], on='IDs', how='left') #5. Merging Our Files Based on ID Number
df_3.to_excel('INMARSAT May 2021 Data.xlsx', index=False) #6. Saving Version to .csv

df_3 #7. Checking My Work


,Bill To:,IDs,Amounts Due,ICCID,MSISDN-C,PI,Type
0,10/11/2019 -,100,13.79,3.000340e+14,3.000340e+14,Sutton,SHORT BURST DATA
1,NaN,101,13.79,3.000340e+14,3.000340e+14,McPhaden,SHORT BURST DATA
2,NaN,102,13.79,3.000340e+14,3.000340e+14,McPhaden,SHORT BURST DATA
3,NaN,103,13.79,3.000340e+14,3.000340e+14,McPhaden,SHORT BURST DATA
4,NaN,104,13.79,3.000340e+14,3.000340e+14,McPhaden,SHORT BURST DATA
...,...,...,...,...,...,...,...
522,NaN,881651461225,15.44,8.988170e+18,8.816938e+11,McPhaden,RUDICS
523,NaN,881651477444,25.19,8.988170e+18,8.816938e+11,Sutton,RUDICS
524,NaN,881651477445,44.94,8.988170e+18,8.816938e+11,Sutton,RUDICS
525,NaN,881651477579,15.44,8.988170e+18,8.816938e+11,Sutton,RUDICS


In [7]:
df_3.sum(axis = 0, skipna = True) #8. Checking Amounts Due Totals

IDs                                              387917450684257
Amounts Due                                              14658.2
ICCID                                                3.90089e+21
MSISDN-C                                             2.64624e+16
PI             SuttonMcPhadenMcPhadenMcPhadenMcPhadenMcPhaden...
Type           SHORT BURST DATASHORT BURST DATASHORT BURST DA...
dtype: object

## Generating PI and Type Billing Report
1. Upload Merged .csv File, 2. Rename Column, 3. Organizing Based on 'PI' and 'Type', 4. Removing Column, 5. Saving Billing Report to .csv, 6. Checking My Work, 7. Checking Amounts Due Totals

In [9]:
df_3 = pd.read_excel('INMARSAT May 2021 Data.xlsx') #1. Upload Merged .csv File

df_3 = df_3.rename(columns={"Amounts Due": "May'21 Amounts Due"}) #2. Rename Column

df_3 = df_3.groupby(['PI', 'Type']).sum() #3. Organizing Based on 'PI' and 'Type'

df_3 = df_3.drop(columns=['IDs','ICCID', 'MSISDN-C']) #4. Removing Column

df_3.to_excel('INMARSAT May 2021 BILL PI TYPES.xlsx') #5. Saving Billing Report to .csv

df_3 #6. Checking My Work

May'21 Amounts Due
PI                                       Type                                
Cronin                                   RUDICS                        402.66
Cross                                    RUDICS                        173.21
                                         SHORT BURST DATA               18.75
Dziak                                    Non-RUDICS                     47.99
                                         RUDICS                         15.44
Feely                                    RUDICS                         96.20
                                         SHORT BURST DATA               13.79
Guyton Eagle Horizon                     RUDICS VOICE                   14.95
Johnson                                  RUDICS                       7554.78
                                         SHORT BURST DATA              247.35
Klinck                                   RUDICS                         15.44
McPhaden                                 RUDICS                       1862.78
                                         SHORT BURST DATA              275.80
Meinig - Dziak                           RUDICS                         30.88
Meinig - Feely                           RUDICS                         30.88
Meinig Chaba                             RUDICS                         15.44
Meinig FCOM1.0                           RUDICS                         15.44
Meinig Former DART                       RUDICS                        234.26
                                         SHORT BURST DATA               27.58
Meinig Former DART DEACTIVATED 4/20/2021 SHORT BURST DATA               46.28
Meinig GNSS GPS                          SHORT BURST DATA               13.79
Meinig SPURS2                            RUDICS                         15.44
Meinig SPURS2 DEACTIVATED 04/20/2021     SHORT BURST DATA               23.14
Meinig pop_up                            RUDICS                        205.72
Mordy IOTAS-ITAE DEACTIVATED 4/20/2021   SHORT BURST DATA              150.41
Mordy ITAE                               RUDICS                        170.45
Stabeno                                  RUDICS                        272.44
                                         SHORT BURST DATA              358.59
Sutton                                   Non-RUDICS                     34.69
                                         RUDICS                       2050.07
                                         SHORT BURST DATA              103.64
Sutton DEACTIVATED 5/06/2021             Non-RUDICS                    119.94

In [10]:
df_3.sum(axis = 0, skipna = True) #7. Checking Amounts Due Totals

May'21 Amounts Due    14658.22
dtype: float64

## Generating PI Billing Report
1. Upload Merged .csv File, 2. Rename Column, 3. Organizing Based on 'PI', 4. Removing Column, 5. Saving Billing Report to .csv, 6. Checking My Work, 7. Checking Amounts Due Totals

In [11]:
df_3 = pd.read_excel('INMARSAT May 2021 Data.xlsx') #1. Upload Merged .csv File

df_3 = df_3.rename(columns={"Amounts Due": "May'21 Amounts Due"}) #2. Rename Column

df_3 = df_3.groupby(['PI']).sum() #3. Organizing Based on 'PI'

df_3 = df_3.drop(columns=['IDs','ICCID', 'MSISDN-C']) #4. Removing Column

df_3.to_excel('INMARSAT May 2021 BILL PI.xlsx') #5. Saving Billing Report to .csv

df_3 #6. Checking My Work

,May'21 Amounts Due
PI,
Cronin,402.66
Cross,191.96
Dziak,63.43
Feely,109.99
Guyton Eagle Horizon,14.95
Johnson,7802.13
Klinck,15.44
McPhaden,2138.58
Meinig - Dziak,30.88


In [12]:
df_3.sum(axis = 0, skipna = True) #7. Checking Amounts Due Totals

May'21 Amounts Due    14658.22
dtype: float64